<a href="https://colab.research.google.com/github/irelandclaisen/Urban_Sprawl/blob/master/Measuring_Urban_Sprawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Measuring Urban Sprawl Over Time

## Description of Project and Strategy

In this notebook, I use a simpler model to measure the amount of urban area in Seattle and Las Vegas in 2020 and in 1984 as a way to measure urban sprawl.  I chose these years based on the availability of Landsat images.  While Landsat 1 has images from 1972, I was unable to get images of the Seattle area from that year.  In fact, I was also unable to get satellite images of the Seattle area from Landsat 2-4.  I was, however, able to get satisfactory satellite images of the Seattle area from Landsat 5, which collected data from 1984 to 2012.  Based on this, I decided that the earliest satellite images I would analyze would be from 1984.  To get more recent data for 2020, I used satellite images from Landsat 8.     

I used land use classification maps drawn by the US Geological Survey to train these simple models.  These maps are done periodically (2001, 2004, 2006, 2008, 2011, 2013 and 2016) and I decided to use the 2016 data and 2016 satellite images from Landsat 8 to train a classification model.  While this model was pretty good at classifying urban areas in 2020 images of Seattle, it did really poorly for the 1984 data.  I looked into it and it seems the way the bands are defined changed between the Landsat 5 and Landsat 8.  To deal with this issue, I trained an additional model using the survey from 2001 and 2001 satellite images from Landsat 5 to classify urban areas in the 1984 images of Seattle. 

Additionally, it seems that models used to classify urban areas in Seattle were not good at classifying areas in Las Vegas.  I suspect this is due to how different the land cover is in Seattle versus Las Vegas, as Las Vegas is largely desert while Seattle has a lot of vegetative areas.  As a results, I made separate classification models for Seattle and Las Vegas.

## Imports and setting up Google Earth Engine

In [ ]:
!pip install geemap

In [3]:
import ee
import geemap

In [4]:
Map = geemap.Map()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=WfZe4vW0Rr_Jb4bJzmo0-JBkIhgKQS6bmBd9-bCgcYg&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g50gmGfXjxMjiei-bw_f9zVOZY0pZ_eWP07NBGlFJFI_nDjCozKYh4

Successfully saved authorization token.


## Methods used in this notebook


In [6]:
def train_classifier(image, area, year):
  """
  Takes in a city image, area (so city geometry) and year.  Uses that information to train a CART classifier
  and returns that classifier   
  """

  # Creates an image of the city overlaid with the land cover label from 2016
  nlcd = ee.Image('USGS/NLCD/NLCD'+str(year)).select('landcover').clip(area)
  # Gets 5000 labeled pixels from the NLCD data
  points = nlcd.sample(**{
      'region': image.geometry(),
      'scale': 30,
      'numPixels': 5000,
      'seed': 0,
      'geometries': True  
  })
  # Use these bands for prediction.
  bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

  # This property of the table stores the land cover labels.
  label = 'landcover'

  # Overlay the points on the image to get training data
  training = image.select(bands).sampleRegions(**{
    'collection': points,
    'properties': [label],
    'scale': 30
  })

  # Train a CART classifier with default parameters.
  classifier = ee.Classifier.smileCart().train(training, label, bands)
  return classifier

In [101]:
def plot_map(image, area, center, zoom = 8):
  """
  Takes in a satellite image and area of interest and prints the image overlaid
  on a map
  """
  Map = geemap.Map()
  # Sets parameters for visualization.  Only show visible bands in image
  vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B2', 'B4', 'B3'],
    'region': area
  }
  Map.centerObject(center, zoom)
  Map.addLayer(image, vis_params)
  return Map 

In [103]:
def plot_land_cover(area, center, year, zoom = 8):
  """
  Takes in an area of interest, center point and year and prints
  the US Geological Survey's land cover map for that area in the given year
  """
  Map = geemap.Map()
  Map.centerObject(center, zoom)
  nlcd = ee.Image('USGS/NLCD/NLCD'+str(year)).select('landcover').clip(area)
  Map.addLayer(nlcd, {}, 'NLCD')
  Map.add_legend(builtin_legend='NLCD')
  return Map

In [104]:
def plot_classified_map(image, area, center, zoom = 8):
  """
  Takes in a image with areas classified by land use and prints the image
  where urban and non urban areas are labeled as red and green respectively
  """
  Map = geemap.Map()
  class_values = [11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 51, 52, 71, 72, 73,
                  74, 81, 82, 90, 95]
  # Converts the colors associated with urban areas to red, water areas to blue 
  # and all other areas to green
  class_palette = ['476ba1', '68ab63', 'ab0000', 'ab0000', 'ab0000', 'ab0000',
                 '68ab63', '68ab63', '68ab63', '68ab63', '68ab63', '68ab63',
                 '68ab63', '68ab63', '68ab63', '68ab63', '68ab63', '68ab63',
                 '68ab63', '68ab63']
  landcover = image.set('classification_class_values', class_values)
  landcover = landcover.set('classification_class_palette', class_palette)
  Map.centerObject(center, zoom)
  Map.addLayer(landcover, {})
  Map.add_legend(labels=['water', 'developed', 'undeveloped'], colors = ['#476ba1', '#ab0000','#68ab63'])
  return Map

In [118]:
def get_area(image, geometry):
  """
  Takes in a image with areas classified as urban and non urban and returns the 
  total area of the urban parts of the image in square kilometers.
  """
  sum_area = 0
  for i in [21, 22, 23, 24]:
    urban = image.eq(i)
    areaImage = urban.multiply(ee.Image.pixelArea());
    area = areaImage.reduceRegion(reducer= ee.Reducer.sum(),
      geometry= geometry,
      scale= 30,
      maxPixels= 1e9
    );
    sum_area += area.get('classification').getInfo()/1000000
  return sum_area

## Seattle

In [22]:
# Area which we are considering the Seattle Greater Area in this project
seattle_AOI = ee.Geometry.Rectangle([237, 46.75, 238.5, 48])
# Center point for Seattle area
seattle_center = ee.Geometry.Point([-122.4439, 47.7538])

### 2016

In [13]:
# Combining satellite images to get a median image of Seattle in 2016
landsat8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
seattle_2016_collection = landsat8.filterBounds(seattle_AOI).filterDate('2016-01-01', '2016-12-31').filterMetadata('CLOUD_COVER', 'less_than', 5)
seattle_2016 = seattle_2016_collection.median().clip(seattle_AOI);

In [74]:
# Satellite image of Seattle for 2016
plot_map(seattle_2016, seattle_AOI, seattle_center)

In [70]:
# Map of Seattle with land cover labels based on the US Geological Survey
plot_land_cover(seattle_AOI, seattle_center, 2016)

In [57]:
# Creates a classifier based on land cover
classifier_2016 = train_classifier(seattle_2016, seattle_AOI, 2016)
# Takes the classifier and applies it to the entire satellite image of Seattle
seattle_2016_labelled = seattle_2016.classify(classifier_2016)

In [65]:
# Prints image of Seattle classified as urban and non urban
plot_classified_map(seattle_2016_labelled, seattle_AOI, seattle_center)

### 2020

In [75]:
# Gets satellite image of Seattle in 2020
seattle_2020_collection = landsat.filterBounds(seattle_AOI).filterDate('2020-01-01', '2020-12-31').filterMetadata('CLOUD_COVER', 'less_than', 5)
seattle_2020 = seattle_2020_collection.median().clip(seattle_AOI);

In [76]:
plot_map(seattle_2020, seattle_AOI, seattle_center)

In [77]:
# Classify areas of Seattle in 2020 as urban or non urban and print the map
seattle_2020_labelled = seattle_2020.classify(classifier_2016)
plot_classified_map(seattle_2020_labelled, seattle_AOI, seattle_center)


In [83]:
print('Urban area in Seattle: ', round(get_area(seattle_2020_labelled, seattle_AOI), 2), 'square kilometers in 2020');

Urban area in Seattle:  3715.93 square kilometers in 2020


### 2001

In [85]:
# Combining satellite images to get a median image of Seattle in 2001
landsat5 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")
seattle_2001_collection = landsat5.filterBounds(seattle_AOI).filterDate('2001-01-01', '2001-12-31').filterMetadata('CLOUD_COVER', 'less_than', 20)
seattle_2001 = seattle_2001_collection.sort('CLOUD_COVER').median().clip(seattle_AOI);

In [86]:
# Satellite image of Seattle for 2001
plot_map(seattle_2001, seattle_AOI, seattle_center)

In [89]:
# Map of Seattle with land cover labels based on the 2001 US Geological Survey
plot_land_cover(seattle_AOI, seattle_center, 2001)

In [90]:
# Creates a classifier based on 2001 land cover
classifier_2001 = train_classifier(seattle_2001, seattle_AOI, 2001)
# Takes the classifier and applies it to the entire satellite image of Seattle
seattle_2001_labelled = seattle_2001.classify(classifier_2001)

In [91]:
# Prints image of 2001 Seattle classified as urban and non urban
plot_classified_map(seattle_2001_labelled, seattle_AOI, seattle_center)

### 1984

In [92]:
# # Combining satellite images to get a median image of Seattle in 1984
seattle_1984_collection = landsat5.filterBounds(seattle_AOI).filterDate('1984-01-01', '1984-12-31').filterMetadata('CLOUD_COVER', 'less_than', 18)
seattle_1984 = seattle_1984_collection.sort('CLOUD_COVER').median().clip(seattle_AOI);

In [93]:
plot_map(seattle_1984, seattle_AOI, seattle_center)

In [94]:
# Classify areas of Seattle in 1984 as urban or non urban and print the map
seattle_1984_labelled = seattle_1984.classify(classifier_2001)
plot_classified_map(seattle_1984_labelled, seattle_AOI, seattle_center)

In [95]:
print('Urban area in Seattle: ', round(get_area(seattle_1984_labelled, seattle_AOI), 2), 'square kilometers in 1984');

Urban area in Seattle:  1479.4 square kilometers in 1984


## Las Vegas

In [98]:
# Area which we are considering the Las Vegas Greater Area in this project
lv_AOI = ee.Geometry.Rectangle([-114.6, 35.6, -115.6, 36.5])
# Center point for the Las Vegas area
lv_center = point = ee.Geometry.Point([-115, 36])

### 2016

In [107]:
# Gets satellite image of Las Vegas in 2016
lv_2016_collection = landsat.filterBounds(lv_AOI).filterDate('2016-01-01', '2016-12-31').filterMetadata('CLOUD_COVER', 'less_than', 5)
lv_2016 = lv_2016_collection.median().clip(lv_AOI);

In [108]:
# Plots map of Las Vegas in 2016
plot_map(lv_2016, lv_AOI, lv_center, zoom = 9)

In [110]:
# Map of Las Vegas with land cover labels based on the US Geological Survey
plot_land_cover(lv_AOI, lv_center, 2016, zoom=9)

In [111]:
# Creates a classifier based on land cover
lv_classifier_2016 = train_classifier(lv_2016, lv_AOI, 2016)
# Takes the classifier and applies it to the entire satellite image of Las Vegas
lv_2016_labelled = lv_2016.classify(lv_classifier_2016)

In [113]:
# Prints image of Las Vegas in 2016 classified as urban and non urban
plot_classified_map(lv_2016_labelled, lv_AOI, lv_center, zoom = 9)

### 2020

In [99]:
# Gets satellite image of Las Vegas in 2020
lv_2020_collection = landsat.filterBounds(lv_AOI).filterDate('2020-01-01', '2020-12-31').filterMetadata('CLOUD_COVER', 'less_than', 5)
lv_2020 = lv_2020_collection.median().clip(lv_AOI);

In [114]:
# Plots map of Las Vegas in 2020
plot_map(lv_2020, lv_AOI, lv_center, zoom = 9)

In [116]:
# Classify areas of Las Vegas in 2020 as urban or non urban and print the map
lv_2020_labelled = lv_2020.classify(lv_classifier_2016)
plot_classified_map(lv_2020_labelled, lv_AOI, lv_center, zoom=9)

In [119]:
print('Urban area in Las Vegas: ', round(get_area(lv_2020_labelled, lv_AOI), 2), 'square kilometers in 2020');

Urban area in Las Vegas:  1156.76 square kilometers in 2020


### 2001

In [120]:
# Gets satellite image of Las Vegas in 2001
lv_2001_collection = landsat5.filterBounds(lv_AOI).filterDate('2001-01-01', '2001-12-31').filterMetadata('CLOUD_COVER', 'less_than', 5)
lv_2001 = lv_2001_collection.sort('CLOUD_COVER').median().clip(lv_AOI);

In [121]:
# Plots map of Las Vegas in 2001
plot_map(lv_2001, lv_AOI, lv_center, zoom = 9)

In [122]:
# Creates a classifier based on land cover
lv_classifier_2001 = train_classifier(lv_2001, lv_AOI, 2001)
# Takes the classifier and applies it to the entire satellite image of Las Vegas
lv_2001_labelled = lv_2001.classify(lv_classifier_2001)

In [123]:
# Prints image of Las Vegas in 2001 classified as urban and non urban
plot_classified_map(lv_2001_labelled, lv_AOI, lv_center, zoom = 9)

### 1984

In [125]:
# Gets satellite image of Las Vegas in 1984
lv_1984_collection = landsat5.filterBounds(lv_AOI).filterDate('1984-01-01', '1984-12-31').filterMetadata('CLOUD_COVER', 'less_than', 5)
lv_1984 = lv_1984_collection.sort('CLOUD_COVER').median().clip(lv_AOI);

In [126]:
# Plots map of Las Vegas in 1984
plot_map(lv_1984, lv_AOI, lv_center, zoom = 9)

In [128]:
# Classify areas of Las Vegas in 1984 as urban or non urban and print the map
lv_1984_labelled = lv_1984.classify(lv_classifier_2001)
plot_classified_map(lv_1984_labelled, lv_AOI, lv_center, zoom=9)

In [129]:
print('Urban area in Las Vegas: ', round(get_area(lv_1984_labelled, lv_AOI), 2), 'square kilometers in 1984');

Urban area in Las Vegas:  470.07 square kilometers in 1984
